# ***simplex method ***

from scratch

In [1]:
#Q1:Implementation of revised simplex method
import numpy as np


def revised_simplex_method(A, b, c):
    m, n = A.shape

    # step 1: initialization
    tableau = np.hstack((A, np.eye(m), b.reshape(-1, 1)))
    c_row = np.hstack((-c, np.zeros(m + 1)))
    tableau = np.vstack((tableau, c_row))



    while np.any(tableau[-1, :-1] < 0):  # Step 3: Feasibility Test
        pivot_col = np.argmin(tableau[-1, :-1])

        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios = np.where(tableau[:-1, pivot_col] > 0, ratios, np.inf)  # Ignore non positive
        pivot_row = np.argmin(ratios)

        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row] /= pivot_element  # Normalizeation

        # Make all other elements in the pivot column zero
        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    solution = np.zeros(n)
    for i in range(n):
        col = tableau[:-1, i]
        if np.sum(col == 1) == 1 and np.sum(col) == 1:
            row = np.where(col == 1)[0][0]
            solution[i] = tableau[row, -1]

    # max obj fun
    F = -tableau[-1, -1]

    # Round the solution and F to the nearest values
    solution = np.round(solution, 2)
    F = round(F, 6)
    return solution, F

A = np.array([[6, 4, 1, 0, 0, 0], [1, 2, 0, 1, 0, 0], [-1, 1, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1]])
b = np.array([24, 6, 1, 2])
c = np.array([5, 4, 0, 0, 0, 0])

solution, F = revised_simplex_method(A, b, c)
x1, x2 = solution[:2]
print( x1,x2,  -F)

3.0 1.5 21.0


<ipython-input-1-34ac8739c279>:18: RuntimeWarning: divide by zero encountered in divide
  ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]


built-in

In [2]:
from scipy.optimize import linprog

c = [-5, -4, 0, 0, 0, 0]

A_eq = [
    [6, 4, 1, 0, 0, 0],  # Constraint 1
    [1, 2, 0, 1, 0, 0],  # Constraint 2
    [-1, 1, 0, 0, 1, 0], # Constraint 3
    [0, 1, 0, 0, 0, 1]   # Constraint 4
]

# Right-hand side vector (b)
b_eq = [24, 6, 1, 2]

x_bounds = [(0, None)] * 6

result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')

x1, x2 = result.x[:2]
F = -result.fun
print( x1,  x2, F)


3.0 1.5 21.0


Two Phase

In [2]:
import numpy as np

def two_phase_simplex_method(c, A, b, artificial_vars_count):
    m, n = A.shape

    A = np.hstack((A, np.eye(m)))
    c_phase1 = np.hstack((np.zeros(n), np.ones(m)))
    B_indices = np.arange(n, n + m)

    tableau = np.hstack((A, b.reshape(-1, 1)))
    c_row = np.hstack((-c_phase1, [0]))
    tableau = np.vstack((tableau, c_row))

    while np.any(tableau[-1, :-1] < 0):
        pivot_col = np.argmin(tableau[-1, :-1])
        ratios = np.divide(tableau[:-1, -1], tableau[:-1, pivot_col],
                           out=np.full(m, np.inf), where=tableau[:-1, pivot_col] > 0)
        pivot_row = np.argmin(ratios)
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row] /= pivot_element

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    if tableau[-1, -1] > 1e-6:
        raise ValueError("The problem is infeasible.")

    tableau = tableau[:, :n + m + 1]
    c_phase2 = np.hstack((c, np.zeros(m)))
    tableau[-1, :-1] = -c_phase2 @ tableau[:-1, :-1] + tableau[-1, :-1]
    tableau[-1, -1] = -c_phase2 @ tableau[:-1, -1] + tableau[-1, -1]

    while np.any(tableau[-1, :-1] < 0):
        pivot_col = np.argmin(tableau[-1, :-1])
        ratios = np.divide(tableau[:-1, -1], tableau[:-1, pivot_col],
                           out=np.full(m, np.inf), where=tableau[:-1, pivot_col] > 0)
        pivot_row = np.argmin(ratios)
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row] /= pivot_element

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    # Extract the solu
    solution = np.zeros(n)
    for i in range(n):
        col = tableau[:-1, i]
        if np.sum(col == 1) == 1 and np.sum(col) == 1:
            row = np.where(col == 1)[0][0]
            solution[i] = tableau[row, -1]

    F = -tableau[-1, -1]

    # Round the solu and F to the nearest reasonable values
    solution = np.round(solution, 6)
    F = round(F, 6)

    return solution, F

c = np.array([4, 1, 0, 0])
A = np.array([
    [3, 1, -1, 0],  # Constraint 1 with artificial var y1
    [4, 3, 0, -1],  # Constraint 2 with artificial var y2
    [1, 2, 0, 0]    # Constraint 3 (no artificial var needed)
])
b = np.array([3, 6, 4])

# Run the Two-Phase Simplex Method
artificial_vars_count = 2  # Number of artificial var added
solution, F = two_phase_simplex_method(c, A, b, artificial_vars_count)
x1, x2 = solution[:2]
print("Optimal solution: x1 =", x1, ", x2 =", x2, ", F =", F)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-a01715a6194f>", line 72, in <cell line: 72>
    solution, F = two_phase_simplex_method(c, A, b, artificial_vars_count)
  File "<ipython-input-3-a01715a6194f>", line 18, in two_phase_simplex_method
    pivot_row = np.argmin(ratios)
  File "/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py", line 1324, in argmin
    kwds = {'keepdims': keepdims} if keepdims is not np._NoValue else {}
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the abo

TypeError: object of type 'NoneType' has no len()

Two phase Built-In

In [4]:
from scipy.optimize import linprog

c = [4, 1]

A_eq = [
    [3, 1, -1, 1, 0],
    [4, 3, 1, 0, 1],
    [1, 2, 0, 0, 0]
]

b_eq = [3, 6, 4]

x_bounds = [(0, None)] * 5

c_phase1 = [0, 0, 0, 1, 1]

result_phase1 = linprog(c_phase1, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')

if result_phase1.success and result_phase1.fun < 1e-6:
    result_phase2 = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=x_bounds, method='highs')

    if result_phase2.success:
        x1, x2 = result_phase2.x[:2]
        F = result_phase2.fun
        print("Optimal solution: x1 =", round(x1, 6), ", x2 =", round(x2, 6), ", F =", round(F, 6))
    else:
        print("not find an optimal solution.")
else:
    print("not find a feasible solution.")


not find a feasible solution.


In [5]:
#Q2:revised simplex method Special cases (alternative_optima)

def revised_simplex_method_with_alternative_optima(A, b, c):
    m, n = A.shape

    # step 1: initialization
    tableau = np.hstack((A, np.eye(m), b.reshape(-1, 1)))  # slack
    c_row = np.hstack((-c, np.zeros(m + 1)))
    tableau = np.vstack((tableau, c_row))

    # Simplex algorithm
    while np.any(tableau[-1, :-1] < 0):
        pivot_col = np.argmin(tableau[-1, :-1])

        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios = np.where(tableau[:-1, pivot_col] > 0, ratios, np.inf)
        pivot_row = np.argmin(ratios)

        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row] /= pivot_element  # Normalizeationa

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    solution = np.zeros(n)
    for i in range(n):
        col = tableau[:-1, i]
        if np.sum(col == 1) == 1 and np.sum(col) == 1:
            row = np.where(col == 1)[0][0]
            solution[i] = tableau[row, -1]

    # max obj fun
    F = -tableau[-1, -1]

    alternative_optima = np.any((tableau[-1, :-1] == 0) & (np.all(tableau[:-1, :-1] == 0, axis=0) == False))

    solution = np.round(solution, 6)
    F = round(F, 6)

    return solution, F, alternative_optima

A = np.array([[1, 2, 1, 0], [1, 1, 0, 1]])
b = np.array([5, 4])
c = np.array([2, 4, 0, 0])

solution, F, alternative_optima = revised_simplex_method_with_alternative_optima(A, b, c)
x1, x2 = solution[:2]
print( x1,  x2,  -F)
if alternative_optima:
    print("Alternative exist")
else:
    print("no alternative")


0.0 2.5 10.0
Alternative exist


In [6]:

def revised_simplex_method_unbounded(A, b, c):
    m, n = A.shape

    tableau = np.hstack((A, np.eye(m), b.reshape(-1, 1)))
    c_row = np.hstack((-c, np.zeros(m + 1)))
    tableau = np.vstack((tableau, c_row))

    while np.any(tableau[-1, :-1] < 0):
        pivot_col = np.argmin(tableau[-1, :-1])

        if np.all(tableau[:-1, pivot_col] <= 0):
            print(" unbounded.")
            return None, None

        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios = np.where(tableau[:-1, pivot_col] > 0, ratios, np.inf)
        pivot_row = np.argmin(ratios)

        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row] /= pivot_element

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    solution = np.zeros(n)
    for i in range(n):
        col = tableau[:-1, i]
        if np.sum(col == 1) == 1 and np.sum(col) == 1:
            row = np.where(col == 1)[0][0]
            solution[i] = tableau[row, -1]

    F = -tableau[-1, -1]

    solution = np.round(solution, 6)
    F = round(F, 6)
    return solution, F

A = np.array([[1, -1, 1, 0], [2, 0, 0, 1]])
b = np.array([10, 40])
c = np.array([2, 1, 0, 0])

solution, F = revised_simplex_method_unbounded(A, b, c)
if solution is not None:
    x1, x2 = solution[:2]
    print( x1,  x2,  F)
else:
    print("unbounded.")


 unbounded.
unbounded.


In [7]:

def revised_simplex_method_infeasible(A, b, c, phase1=True):
    m, n = A.shape

    if phase1:
        c = np.zeros(n)
        c[-1] = -1

    tableau = np.hstack((A, b.reshape(-1, 1))).astype(float)
    c_row = np.hstack((-c, 0))
    tableau = np.vstack((tableau, c_row))

    while np.any(tableau[-1, :-1] < 0):
        pivot_col = np.argmin(tableau[-1, :-1])

        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios = np.where(tableau[:-1, pivot_col] > 0, ratios, np.inf)
        if np.all(ratios == np.inf):
            print(" infeasible.")
            return None, None

        pivot_row = np.argmin(ratios)
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row] /= pivot_element

        for i in range(len(tableau)):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    if phase1:
        if abs(tableau[-1, -1]) > 1e-6:
            print(" infeasible.")
            return None, None

    return tableau[:-1, -1], -tableau[-1, -1]

A = np.array([[2, 1, 1, 0, 0], [3, 4, 0, -1, 1]])
b = np.array([2, 12])
c = np.array([3, 2, 0, 0, 0])

solution, F = revised_simplex_method_infeasible(A, b, c, phase1=True)
if solution is not None:
    print("Feasible solution found")
    solution, F = revised_simplex_method_infeasible(A, b, c, phase1=False)
    if solution is not None:
        x1, x2 = solution[:2]
        print( x1, x2,  -F)
else:
    print(" infeasible.")


Feasible solution found
2.0 4.0 4.0
